In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from pytorchcv.model_provider import get_model
import torchvision.models as models

In [2]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [3]:
cache = "/scratch/doa240/.cache/datasets"
train_data = torchvision.datasets.CIFAR10(root=cache, train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(train_data, batch_size=4, shuffle=True, num_workers=4)

Files already downloaded and verified


In [4]:
classes = ('Airplane', 'Car', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck')

In [5]:
model = models.vgg19()
# model = get_model('vgg19', pretrained=False, root="/scratch/doa240/.torch/models")

In [6]:
import torch.nn as nn
model.classifier[3] = nn.Linear(4096,1024)
model.classifier[6] = nn.Linear(1024,10)
# model.output.fc2.fc = nn.Linear(4096,1024)
# model.output.fc3 = nn.Linear(1024,10)
model.train()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [8]:
model.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [9]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [10]:
import time

In [12]:
for epoch in range(20):  # loop over the dataset multiple times
    print("Epoch", epoch,"...............")
    running_loss = 0.0
    start_time = time.time()
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        output = model(inputs)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()

        #Time
        end_time = time.time()
        time_taken = end_time - start_time

        # print statistics
        running_loss += loss.item()
        if (i+1) % 5000 == 0:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
            print('Time:',time_taken)
            running_loss = 0.0

print('Finished Training of AlexNet')

Epoch 0 ...............
[1,  5000] loss: 4.134
Time: 191.3513207435608
[1, 10000] loss: 3.608
Time: 387.1475932598114
Epoch 1 ...............
[2,  5000] loss: 2.806
Time: 185.37382769584656
[2, 10000] loss: 2.519
Time: 370.4654395580292
Epoch 2 ...............
[3,  5000] loss: 2.070
Time: 185.64786529541016
[3, 10000] loss: 2.042
Time: 371.03962421417236
Epoch 3 ...............
[4,  5000] loss: 1.639
Time: 185.7948455810547
[4, 10000] loss: 1.663
Time: 371.31680059432983
Epoch 4 ...............
[5,  5000] loss: 1.307
Time: 185.91152215003967
[5, 10000] loss: 1.343
Time: 371.8131287097931
Epoch 5 ...............
[6,  5000] loss: 0.990
Time: 185.84895205497742
[6, 10000] loss: 1.087
Time: 371.4946095943451
Epoch 6 ...............
[7,  5000] loss: 0.789
Time: 185.56001019477844
[7, 10000] loss: 0.852
Time: 370.3558580875397
Epoch 7 ...............
[8,  5000] loss: 0.632
Time: 184.96944952011108
[8, 10000] loss: 0.686
Time: 370.0859637260437
Epoch 8 ...............
[9,  5000] loss: 0.461
T

In [13]:
test_data = torchvision.datasets.CIFAR10(root=cache, train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(test_data, batch_size=4, shuffle=False, num_workers=4)

Files already downloaded and verified


In [14]:
#Testing Accuracy
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %.2f %%' % (100 * correct / total))

Accuracy of the network on the 10000 test images: 79.59 %


In [15]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %.3f %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of Airplane : 81.900 %
Accuracy of   Car : 89.100 %
Accuracy of  Bird : 71.900 %
Accuracy of   Cat : 63.400 %
Accuracy of  Deer : 78.500 %
Accuracy of   Dog : 70.300 %
Accuracy of  Frog : 83.300 %
Accuracy of Horse : 83.800 %
Accuracy of  Ship : 88.800 %
Accuracy of Truck : 85.700 %


In [16]:
# #Verifying average accuracy of the network
# avg = 0
# for i in range(10):
#   temp = (100 * class_correct[i] / class_total[i])
#   avg = avg + temp
# avg = avg/10
# print('Average accuracy = ', avg)  